<center>
    <h1>Building Custom Jupyter Magics</h1>
<img src='jupyter_logo.png' style='width: 300px; padding-top: 100px'>
</center>

## Part 1: Built-In Magics

In [ ]:
import numpy as np
import pandas as pd
import requests
from string import ascii_lowercase

from confundo.binary import search_for_metric
from htools import magics
import pandas_htools

In [ ]:
%who

In [ ]:
%%timeit -n 10 -r 5
arr = []
for i in range(1_000_000):
    arr.append(i)

In [ ]:
%%writefile spam.py
def spam(n):
    return 'spam' * n


if __name__ == '__main__':
    print(spam(5))


In [ ]:
%pycat spam.py
!rm spam.py

## Part 2: Custom Magics

In [ ]:
df = pd.DataFrame(np.random.randint(0, 10, (6, 10)), 
                  columns=list(ascii_lowercase)[:10])

In [ ]:
df.shape
df.head(2)
df.tail(2)

In [ ]:
%%talk
df.shape
df.ends(2)

In [ ]:
df[['a', 'b']] = np.nan
[np.nanmean(df[col]) for col in df.columns]

In [ ]:
%%lax
df['a'] = np.nan
df['b'] = np.nan
[np.nanmean(df[col]) for col in df.columns]

In [ ]:
%%nag
df['a'] = np.nan
df['b'] = np.nan
[np.nanmean(df[col]) for col in df.columns]

## Part 3:  Building Custom Magics

In [ ]:
from IPython.core.magic import cell_magic, magics_class, Magics
from IPython.core.magic_arguments import (argument, magic_arguments,
                                          parse_argstring)
from operator import itemgetter

from htools import timebox, TimeExceededError, timebox_handler

In [ ]:
preds = pd.read_csv('sample_predictions.csv')
preds.head()

In [ ]:
thresholds = dict()
for thresh in (.5, .75, .9, .95, .99):
    thresholds[thresh] = search_for_metric('precision', thresh, 
                                           preds.y_true, preds.y_proba)

In [ ]:
@magics_class
class TimeboxMagic(Magics):

    @cell_magic
    def timebox(self, line=None, cell=None):
        print('Line: ' + line)
        print('\nCell:\n' + cell)

In [ ]:
get_ipython().register_magics(TimeboxMagic)

In [ ]:
%%timebox
nums = list(range(5))
print(nums)

In [ ]:
%%timebox 123
nums = list(range(5))
print(nums)

In [ ]:
@magics_class
class TimeboxMagic(Magics):

    @cell_magic
    def timebox(self, line=None, cell=None):
        print('Line: ' + line)
        print('\nCell:\n' + cell)
        self.shell.run_cell(cell)

In [ ]:
get_ipython().register_magics(TimeboxMagic)

In [ ]:
%%timebox
nums = list(range(5))
print(nums)

In [ ]:
@magics_class
class TimeboxMagic(Magics):

    @cell_magic
    @magic_arguments()
    @argument('time', type=int,
              help='Max number of seconds before throwing error.')
    @argument('-p', action='store_true',
              help='Boolean flag: if provided, use permissive '
                   'execution (if the cell exceeds the specified '
                   'time, no error will be thrown, meaning '
                   'following cells can still execute.) If '
                   'flag is not provided, default behavior is to '
                   'raise a TimeExceededError and halt notebook '
                   'execution.')
    def timebox(self, line=None, cell=None):
        args = parse_argstring(self.timebox, line)
        print('Line: ' + line)
        print('\nCell:\n' + cell)
        print('Args:', args)
        self.shell.run_cell(cell)

In [ ]:
get_ipython().register_magics(TimeboxMagic)

In [ ]:
%%timebox 5
nums = list(range(5))
print(nums)

In [ ]:
%%timebox 3 -p
nums = list(range(5))
print(nums)

In [ ]:
@magics_class
class TimeboxMagic(Magics):
    """Timebox a cell's execution to a user-specified duration. As with any
    standard try/except block, note that values can change during execution
    even if an error is eventually thrown (i.e. no rollback occurs).
    
    Sample usage:
    
    %%timebox 3
    # Throw error if cell takes longer than 3 seconds to execute.
    output = slow_function(*args)
    
    %%timebox 3 -p
    # Attempt to execute cell for 3 seconds, then give up. Message is printed
    # stating that time is exceeded but no error is thrown.
    output = slow_function(*args)
    """

    @cell_magic
    @magic_arguments()
    @argument('time', type=int,
              help='Max number of seconds before throwing error.')
    @argument('-p', action='store_true',
              help='Boolean flag: if provided, use permissive '
                   'execution (if the cell exceeds the specified '
                   'time, no error will be thrown, meaning '
                   'following cells can still execute.) If '
                   'flag is not provided, default behavior is to '
                   'raise a TimeExceededError and halt notebook '
                   'execution.')
    def timebox(self, line=None, cell=None):
        args = parse_argstring(self.timebox, line)
        with timebox(args.time) as tb:
            if args.p:
                cell = self._make_cell_permissive(cell)
            self.shell.run_cell(cell)

    @staticmethod
    def _make_cell_permissive(cell):
        """Place whole cell in try/except block."""
        robust_cell = (
            'try:\n\t' + cell.replace('\n', '\n\t')
            + '\nexcept:\n\tprint("Time exceeded. '
            '\\nWarning: objects may have changed during execution.")'
        )
        return robust_cell

In [ ]:
get_ipython().register_magics(TimeboxMagic)

In [ ]:
timebox??

In [ ]:
timebox_handler??

In [ ]:
%%lax
%%timebox 3

thresholds = dict()
for thresh in (.5, .75, .9, .95, .99):
    thresholds[thresh] = search_for_metric('precision', thresh, 
                                           preds.y_true, preds.y_proba)

In [ ]:
%%lax
%%timebox 6 -p

thresholds = dict()
for thresh in (.5, .75, .9, .95, .99):
    thresholds[thresh] = search_for_metric('precision', thresh, 
                                           preds.y_true, preds.y_proba)

In [ ]:
print('Proceeding to next cell.\n')
print('Precision = {} \nThreshold = {:.3f}'.format(*max(thresholds.items(), 
                                                   key=itemgetter(0))))

## Sources

Documentation: https://ipython.readthedocs.io/en/stable/config/custommagics.html

IPython github: https://github.com/ipython/ipython/tree/master/IPython/core

slides/notebook: https://github.com/hdmamin/custom-jupyter-magics

image source: https://jupyter.org/assets/main-logo.svg